In [1]:
import numpy as np
from tokenizers import ByteLevelBPETokenizer
import onnxruntime as ort

In [2]:
# Load the ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer.from_file(
    vocab_filename='./tokenizer/vocab.json',
    merges_filename='./tokenizer/merges.txt'
)

In [3]:
# Load the ONNX model
onnx_model_path = "./QAT/QAT_int_8.onnx"  # Path to your INT8-quantized ONNX model
session = ort.InferenceSession(onnx_model_path)

In [4]:
# Function to preprocess and predict sentiment for a custom review
def predict_sentiment(review):
    # Tokenize the review
    review_sequence = tokenizer.encode(review).ids
    
    # Pad the sequence
    max_length = 170  # Maximum sequence length (same as original)
    pad_token_id = tokenizer.token_to_id("<pad>")
    review_padded = np.pad(
        review_sequence[:max_length], 
        (0, max_length - len(review_sequence[:max_length])), 
        mode='constant', 
        constant_values=pad_token_id
    ).astype(np.float32)  # ONNX expects float32 input
    
    # Reshape for ONNX (batch_size, sequence_length)
    review_padded = review_padded.reshape(1, -1)  # Shape: (1, 170)

    # Get input name for ONNX model
    input_name = session.get_inputs()[0].name
    
    # Predict sentiment
    prediction = session.run(None, {input_name: review_padded})[0]  # Output shape: (1, 1)
    
    # Determine sentiment based on prediction
    sentiment = "Positive" if prediction[0][0] > 0.44 else "Negative"
    
    return sentiment, prediction

In [5]:
# Test with a custom review
custom_review = "I didn't like it. Others seem to love it though"
sentiment, prediction = predict_sentiment(custom_review)
print(f"Custom Review Sentiment: {sentiment}, Prediction Score: {prediction[0][0]:.4f}")

Custom Review Sentiment: Negative, Prediction Score: 0.2979
